In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib import rc # 폰트
from datetime import datetime
import time
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')
# datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')

path = 'C:\\Users\\wldbs\\Desktop\\kart\\1031\\'

In [22]:
raw_kpi = pd.read_csv(path + "00-single_speed_09121010-kpi.csv")
#raw_meta = pd.read_csv(path + "00-json-metadata-trackId-with-map-name.csv", encoding = 'cp949')
crawl = pd.read_csv(path + "00_crawl_data_with_theme.csv")
#raw_meta

In [23]:
kpi = pd.merge(crawl, raw_kpi, left_on = 'trackId', right_on = 'track_id', how = 'right')
kpi.head(5)

,map_name,난이도,랩,트랙 길이,진행 방향,첫 등장일,년,월,일,출시일,테마,trackId,track_id,cnt_match,cnt_user,cnt_retire_user,percent_retire,SUM_record,AVG_record
0,빌리지 운하,1.0,3.0,4.8,시계방향,2004년 6월 1일,2004.0,6.0,1.0,2004-06-01 00:00:00,빌리지,3a801d3a1be83ef6c290f942e9a9c90b8a5ebc8d978eaa...,3a801d3a1be83ef6c290f942e9a9c90b8a5ebc8d978eaa...,10325,215222,70792,32.8925,1809053528,51731.585016
1,트랙 9,2.0,1.0,3.5,단선형,2013년 8월 8일,2013.0,8.0,8.0,2013-08-08 00:00:00,해변,973116ef458e79c7a89e3c4a052472e47e06d65fc84d9c...,973116ef458e79c7a89e3c4a052472e47e06d65fc84d9c...,7067,12807,3766,29.4058,457084946,55451.285454
2,노르테유 허공의 갈림길,2.0,3.0,3.9,반시계방향,2005년 8월 25일,2005.0,8.0,25.0,2005-08-25 00:00:00,노르테유,caf2382bb9c0cf432379d97e5658cd0f1f7ad0008bac0a...,caf2382bb9c0cf432379d97e5658cd0f1f7ad0008bac0a...,5343,8232,1794,21.7930,346111840,58982.931152
3,코리아 제주 해오름 다운힐,2.0,1.0,5.6,단선형,2021년 8월 12일,2021.0,8.0,12.0,2021-08-12 00:00:00,코리아,b57588a6c4fd89fbc2a2c9e9c1a95a746c35ab3ee08e39...,b57588a6c4fd89fbc2a2c9e9c1a95a746c35ab3ee08e39...,4183,25468,8688,34.1134,496217651,68794.905171
4,빌리지 고가의 질주,2.0,2.0,7.4,시계방향,2004년 6월 1일,2004.0,6.0,1.0,2004-06-01 00:00:00,빌리지,7394f4ea05d4115bc1308101998132e50e366c4456c636...,7394f4ea05d4115bc1308101998132e50e366c4456c636...,2714,30486,10096,33.1168,780886822,116969.266327


In [24]:
# track_name 기준으로 null 제거
kpi.map_name.isnull().sum()
kpi = kpi.dropna(subset=['map_name'])

In [25]:
kpi.isnull().sum()

map_name            0
난이도                37
랩                  37
트랙 길이              37
진행 방향              37
첫 등장일              37
년                  37
월                  37
일                  37
출시일                37
테마                  3
trackId             0
track_id            0
cnt_match           0
cnt_user            0
cnt_retire_user     0
percent_retire      0
SUM_record          0
AVG_record          0
dtype: int64

In [26]:
# 하루에 한 번도 플레이 하지 않은 맵 (4일 동안 최소 4번, n일동안 최소 n번) 은 삭제하기로 함

kpi_for_match = kpi
cnt_match_df = kpi_for_match.loc[kpi.cnt_match > 28]

In [27]:
# 28회가 전체의 몇 퍼센트 정도에 위치하는지 확인
# 145개 / 전체 319개 맵
kpi_for_match.loc[kpi_for_match.cnt_match <= 28]

print(len(kpi_for_match.loc[kpi_for_match.cnt_match <= 28])/len(kpi_for_match) * 100)   # 약 하위 45.45%를 다 날려버리고 시작하게 됨

45.45454545454545


In [28]:
# 이상치 확인 함수
def detect_num(df):
    for i in df.select_dtypes(exclude='object').columns:
        Q1 = df[i].quantile(0.25)
        Q3 = df[i].quantile(0.75)
        IQR = Q3 - Q1
        print( i, df[i][ (df[i]>Q3 + IQR*1.5) | (df[i]<Q1 - IQR*1.5) ].count() )
        print( i, 'Q1 이하: ', df[i][ (df[i]<Q1 - IQR*1.5) ].count() )
        print( i, 'Q3 이상: ', df[i][ (df[i]>Q3 + IQR*1.5) ].count() )
        print( i, '이상치의 전체 비율: ', round(df[i][ (df[i]>Q3 + IQR*1.5) | (df[i]<Q1 - IQR*1.5) ].count() / df[i].count(), 5) * 100 )
        print("="*36)

In [29]:
# 전체 매치수

cnt_match_df.cnt_match.sum()

79949

In [30]:
# 사용률 백분위 컬럼 생성

cnt_match_df['percentage'] = round(cnt_match_df.cnt_match/cnt_match_df.cnt_match.sum()*100, 3)

In [31]:
# 로그 씌우기 전 cnt값 사분위수 확인하기

Q1 = cnt_match_df['cnt_match'].quantile(0.25)
Q2 = cnt_match_df['cnt_match'].quantile(0.5)
Q3 = cnt_match_df['cnt_match'].quantile(0.75)

print('Q1 : ', Q1)
print('Q2 : ', Q2)
print('Q3 : ', Q3)

print('median : ', cnt_match_df.cnt_match.median())

Q1 :  59.25
Q2 :  128.0
Q3 :  482.75
median :  128.0


In [32]:
# 이상치 갯수 확인

detect_num(cnt_match_df)

난이도 0
난이도 Q1 이하:  0
난이도 Q3 이상:  0
난이도 이상치의 전체 비율:  0.0
랩 2
랩 Q1 이하:  0
랩 Q3 이상:  2
랩 이상치의 전체 비율:  1.307
트랙 길이 4
트랙 길이 Q1 이하:  1
트랙 길이 Q3 이상:  3
트랙 길이 이상치의 전체 비율:  2.614
년 2
년 Q1 이하:  2
년 Q3 이상:  0
년 이상치의 전체 비율:  1.307
월 0
월 Q1 이하:  0
월 Q3 이상:  0
월 이상치의 전체 비율:  0.0
일 0
일 Q1 이하:  0
일 Q3 이상:  0
일 이상치의 전체 비율:  0.0
cnt_match 11
cnt_match Q1 이하:  0
cnt_match Q3 이상:  11
cnt_match 이상치의 전체 비율:  6.322
cnt_user 29
cnt_user Q1 이하:  0
cnt_user Q3 이상:  29
cnt_user 이상치의 전체 비율:  16.667
cnt_retire_user 27
cnt_retire_user Q1 이하:  0
cnt_retire_user Q3 이상:  27
cnt_retire_user 이상치의 전체 비율:  15.517
percent_retire 6
percent_retire Q1 이하:  0
percent_retire Q3 이상:  6
percent_retire 이상치의 전체 비율:  3.4479999999999995
SUM_record 28
SUM_record Q1 이하:  0
SUM_record Q3 이상:  28
SUM_record 이상치의 전체 비율:  16.092000000000002
AVG_record 6
AVG_record Q1 이하:  0
AVG_record Q3 이상:  6
AVG_record 이상치의 전체 비율:  3.4479999999999995
percentage 11
percentage Q1 이하:  0
percentage Q3 이상:  11
percentage 이상치의 전체 비율:  6.322


### 상위 트랙 

In [35]:
# (1) 상위 트랙 기준 정하기 (상위 25%)
# Q3 이상, 상위 테이블로 만들기
# 482.75이상, 44건

upper_Q3_df = cnt_match_df.loc[cnt_match_df.cnt_match >= Q3]
print(len(upper_Q3_df))
#print(Q3)upper_Q3_df
#display(upper_Q3_df)

44


In [37]:
# 상위 트랙 결측치 확인

upper_Q3_df.isnull().sum()
upper_Q3_df[upper_Q3_df['난이도'].isnull()==True]

,map_name,난이도,랩,트랙 길이,진행 방향,첫 등장일,년,월,일,출시일,테마,trackId,track_id,cnt_match,cnt_user,cnt_retire_user,percent_retire,SUM_record,AVG_record,percentage
6,사막 버려진 오아시스,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,사막,bc4cc07b793a1734626a114d9e9527255d2a0dc56cfe0f...,bc4cc07b793a1734626a114d9e9527255d2a0dc56cfe0f...,1543,7449,3560,47.7916,219874298,104354.199336,1.930
7,메이플 헤네시스 공원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,메이플,0e324e952312814d287d63fa6f6bdee65fca748fbae2ac...,0e324e952312814d287d63fa6f6bdee65fca748fbae2ac...,1507,20226,7559,37.3727,379016080,98933.980684,1.885
9,메이플 리스항구 언덕길,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,메이플,73103a17062d7007e64739fefc2aee4ba0d98baa53c1f9...,73103a17062d7007e64739fefc2aee4ba0d98baa53c1f9...,1452,19738,8864,44.9083,316402232,92515.272515,1.816
13,메이플 레헬른 악몽의 시계탑,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,메이플,c81f0b80597f0d4681dd925c08f9250512a7dd8dd92572...,c81f0b80597f0d4681dd925c08f9250512a7dd8dd92572...,1079,15722,11523,73.2922,337434196,136668.366140,1.350
16,메이플 리스항구 선착장,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,메이플,b7cb8cf6cfe163b27b7cde2a66de24394bb51afc5102d0...,b7cb8cf6cfe163b27b7cde2a66de24394bb51afc5102d0...,1030,1954,369,18.8843,96160664,75717.058268,1.288
20,노르테유 스카이웨이,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,노르테유,12170098c74c9d6d7db9abb0d5e0cde38c685b5c171f1d...,12170098c74c9d6d7db9abb0d5e0cde38c685b5c171f1d...,852,14073,8563,60.8470,301792882,143369.540143,1.066
40,사막 베거이 시장,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,사막,6b4df446275a17345901a06f1e89ae7e2269c81b63d548...,6b4df446275a17345901a06f1e89ae7e2269c81b63d548...,509,5031,1877,37.3087,110746588,92674.969038,0.637


### 하위 트랙

In [34]:
# (1) 하위 트랙 기준 정하기, 하위 25%
# Q1 미만을 확인, 44개
# 29 ~ 59

lower_Q1_df = cnt_match_df.loc[cnt_match_df.cnt_match < Q1]
len(lower_Q1_df)
# display(lower_Q1_df)

44

In [38]:
# 하위 트랙 결측치 확인

lower_Q1_df.isnull().sum()
lower_Q1_df[lower_Q1_df['난이도'].isnull()==True]

,map_name,난이도,랩,트랙 길이,진행 방향,첫 등장일,년,월,일,출시일,테마,trackId,track_id,cnt_match,cnt_user,cnt_retire_user,percent_retire,SUM_record,AVG_record,percentage
138,광산의 지배자,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,광산,03f7cfe4dc950329d0fd519fe1c5755b54582b30ebed33...,03f7cfe4dc950329d0fd519fe1c5755b54582b30ebed33...,52,425,207,48.7059,33607967,373421.855556,0.065
141,[R] 사막 빙글빙글 공사장,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b925d9e4efc94e1f61c300340eab1db324dc80476d73ff...,b925d9e4efc94e1f61c300340eab1db324dc80476d73ff...,49,643,788,122.5505,20080212,198813.980198,0.061
149,마비노기 바올 던전,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,마비노기,a9a8be547d729112e02f1301f2b6a43ee3cbdcaad32c3d...,a9a8be547d729112e02f1301f2b6a43ee3cbdcaad32c3d...,43,478,488,102.0921,12617214,124922.910891,0.054
150,대해적 로두마니,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해적,6c9a71c4d3f843af9bf3f129cf9f4a213cd79f73db8f01...,6c9a71c4d3f843af9bf3f129cf9f4a213cd79f73db8f01...,42,478,720,150.6276,23651700,307164.935065,0.053
151,마비노기 던바튼,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,마비노기,f62450db52ad3875acb8eceeba700f861fa3e510e316a0...,f62450db52ad3875acb8eceeba700f861fa3e510e316a0...,42,172,19,11.0465,6407112,92856.695652,0.053


### 저장

In [39]:
# 저장

upper_Q3_df.to_csv(path + 'find_null_upper_map.csv', index=False, encoding = 'utf-8')

In [40]:
# 저장

lower_Q1_df.to_csv(path + 'find_null_lower_map.csv', index=False, encoding = 'utf-8')